In [1]:
# load data
# import torch
# from torch_geometric.data import Data
# from torch_geometric.nn import GCNConv, GNNExplainer
import scipy.io as sio

transaction_mat = sio.loadmat("dataset/transaction.mat")

transactions = transaction_mat["transactions"] #1000笔交易 eg: [[ 526  589  304],...] 可能是债权人 债务人 资金提供方

transaction_label = transaction_mat["transaction_label"][0] #size:1000笔交易的标签

transaction_feature = transaction_mat["transaction_feature"][0] #size:1000笔交易的特征

matrix_transaction = transaction_mat["matrix_transaction"] #1000*1000的向量表示 matrix_transaction[i][j]>0 表示有交易

transaction_sources = []
transaction_targets = []
for i in range(len(transaction_label)):
    for j in range(len(transaction_label)):
        if matrix_transaction[i][j]>0:
            transaction_sources.append(i)
            transaction_targets.append(j)
# 特征            
features = [[0]]*8554            
for a in range(len(transactions)):
    i = transactions[a][0]
    j = transactions[a][1]
    k = transactions[a][2]

    features[i] = [transaction_feature[a]]
    features[j] = [transaction_feature[a]]
    features[k] = [transaction_feature[a]]
# transaction_datas = pd.DataFrame({'transaction_id':range(len(transaction_label)),'risks':transaction_feature,'subject':transaction_label})
# transaction_edges =  pd.DataFrame({'source':transaction_sources,'target':transaction_targets})

# 标签
labels = [[0]]*8554
for a in range(len(transactions)):
    i = transactions[a][0]
    j = transactions[a][1]
    k = transactions[a][2]

    labels[i] = [transaction_label[a]]
    labels[j] = [transaction_label[a]]
    labels[k] = [transaction_label[a]]


In [2]:
# 处理data：处理边-edge_index  处理节点特征-x
# 处理节点标签
import torch
from torch_geometric.data import Data


edge_index = torch.tensor([transaction_sources,transaction_targets], dtype=torch.long)


# TODO:每个节点增加特征，非必需

x = torch.tensor(features, dtype=torch.float)
y = torch.tensor(labels, dtype=torch.float)

data = Data(edge_index=edge_index,x=x,y=y)

print(data)

Data(x=[8554, 1], edge_index=[2, 8554], y=[8554, 1])


In [3]:
import torch.nn.functional as F

from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
#         self.conv1 = GCNConv(dataset.num_node_features, 1)
#         self.conv2 = GCNConv(16, dataset.num_classes)
        self.conv1 = GCNConv(1, 16)
        self.conv2 = GCNConv(16, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(20):
    optimizer.zero_grad()
    out = model(data)
    print(out)
#     TODO
    loss = F.nll_loss(out, data.y) 
    loss.backward()
    optimizer.step()

Net(
  (conv1): GCNConv(1, 16)
  (conv2): GCNConv(16, 2)
)

In [14]:
import matplotlib.pyplot as plt
from torch_geometric.nn import GNNExplainer

x, edge_index = data.x, data.edge_index

explainer = GNNExplainer(model, epochs=30)
node_idx = 12
node_feat_mask, edge_mask = explainer.explain_node(node_idx, x, edge_index)
ax, G = explainer.visualize_subgraph(node_idx, edge_index, edge_mask, y=data.y)
plt.show()

Data(x=[8554, 1], edge_index=[2, 8554], y=[8554, 1])


TypeError: forward() takes 2 positional arguments but 3 were given